In [ ]:
import numpy as np

from skorecard import datasets
from skorecard.preprocessing import SimpleBucketTransformer
from skorecard.preprocessing import AgglomerativeBucketTransformer
from skorecard.preprocessing import QuantileBucketTransformer
from skorecard.preprocessing import TreeBucketTransformer

import pandas as pd

In [ ]:
df = datasets.load_uci_credit_card(as_frame=True)
df.head(10)


### Simple Example

In [ ]:
X = df['MARRIAGE'].values

In [ ]:
MyBucketTransformer = AgglomerativeBucketTransformer(bin_count=2)

In [ ]:
MyBucketTransformer.fit(X)

In [ ]:
MyBucketTransformer.transform(X)

In [ ]:
MyBucketTransformer.BucketDict

### Practical Example with Feature Union

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from skorecard.pipeline import ColumnSelector

In [ ]:
transformer = Pipeline([
    ('bucketed_features', FeatureUnion(n_jobs=1, transformer_list=[

        ('simple', Pipeline([
            ('selector', ColumnSelector(['EDUCATION'])),
            ('bucketer', SimpleBucketTransformer(bin_count=4)),
            
        ])), 
        
        ('agglom', Pipeline([
            ('selector', ColumnSelector(['MARRIAGE'])),
            ('bucketer', AgglomerativeBucketTransformer(bin_count=2)),
        ])),

        ('quantile', Pipeline([
            ('selector', ColumnSelector(['LIMIT_BAL'])),
            ('bucketer', QuantileBucketTransformer(bin_count=10)),
        ])), 
        
        ('dummy', Pipeline([
            ('selector', ColumnSelector(['BILL_AMT1', 'default'])),
        ]))

    ])),  # bucketed_features close
])  # pipeline close


In [ ]:
transformer.fit_transform(df)

### Practical Examples with ColumnTransformer

In [ ]:
from skorecard.preprocessing import ManualBucketTransformer
from sklearn.compose import ColumnTransformer 

In [ ]:
df = datasets.load_uci_credit_card(as_frame=True)
transformer = ColumnTransformer(
    transformers=[
        ('simple', SimpleBucketTransformer(bin_count=2), [1]),
        ('agglom', AgglomerativeBucketTransformer(bin_count=4), [0]),
        ('quantile', QuantileBucketTransformer(bin_count=[10, 6]), [2, 3])
    ],
    remainder='passthrough'
)

In [ ]:
transformer.fit_transform(df.values)

In [ ]:
transformer.named_transformers_.quantile.BucketDict

In [ ]:
transformer.named_transformers_

### Manual Bucket Transformer

In [ ]:
example_boundary_dict = {}

for i in range(len(transformer.named_transformers_) - 1):
    key = list(transformer.named_transformers_.keys())[i]
    for j in range(len(transformer.transformers[i][2])):
        example_boundary_dict[transformer.transformers[i][2][j]]= transformer.named_transformers_[key].BucketDict[f'Feature_{j}'].boundaries

In [ ]:
example_boundary_dict

In [ ]:
MBT = ManualBucketTransformer(boundary_dict=example_boundary_dict)

In [ ]:
X = df.copy()
MBT.fit(X.values)
a = MBT.transform(X.values)

In [ ]:
np.unique(a[:,3])

### Tree Bucket Transformer

In [ ]:
from skorecard.preprocessing import TreeBucketTransformer

In [ ]:
X = df['LIMIT_BAL'].values
y = df['default'].values

In [ ]:
tbt = TreeBucketTransformer(inf_edges=False, 
    max_depth=2,
   criterion = 'entropy',
   min_samples_leaf =2000 , #Minimum number of entries in the bins
   min_impurity_decrease=0.001)

In [ ]:
tbt.fit(X, y)

In [ ]:
tbt.transform(X)

### Categorical Transformer

In [ ]:
from skorecard.preprocessing import CatBucketTransformer

In [ ]:
X = df['EDUCATION'].values
y = df['default'].values

In [ ]:
sum(df['EDUCATION'].value_counts(normalize=True) > 0.2)

In [ ]:
np.unique(X)

In [ ]:
cbt = CatBucketTransformer(threshold_min=0.01, threshold_max=0.2, epsilon=0.05)

In [ ]:
cbt.fit(X, y)

In [ ]:
X = cbt.transform(X)

In [ ]:
np.unique(X)

In [ ]:
cbt.mapping_dict